# Implementing Iteration

## Agenda

1. Review: Iteration
2. Details: *iterables*, *iterators*, `iter`, and `next`
3. Implementing iterators with classes
4. Implementing iterators with *generators* and `yield`
    - lazy evalulation/ coroutine behavior 

## 1. Iteration

*Iteration* simply refers to the process of accessing — one by one — the items stored in some container (higher level category of some data structure thing, set, list, dict, whatever). The order of the items, and whether or not the iteration is comprehensive, depends on the container.

In Python, we typically perform iteration using the `for` loop.

In [1]:
for i in range(10): #range is a sequence and range is ordered
    print(i)

0
1
2
3
4
5
6
7
8
9


In [2]:
for c in 'CS 331': #another ordered sequence
    print(c)

C
S
 
3
3
1


In [3]:
for e in [3.14, 'hello', True]: #lists are ordered
  print(e)

3.14
hello
True


In [4]:
for n in {5, 6, 7, 5, 3, 0, 9}: #sets are not ordered
  print(n)

0
3
5
6
7
9


In [5]:
for k,v in {'a': 'aardvark', 'b': 'bear', 'c': 'cat'}.items(): #get back a list of tuples 
    print(f'{k} -> {v}')

a -> aardvark
b -> bear
c -> cat


## 2. *iterables*, *iterators*, `iter`, and `next`

We can iterate over anything that is *iterable*. Intuitively, if something can be used as the source of items in a `for` loop, it is iterable.

But how does a `for` loop really work?

In [18]:
l = ['iteration', 'is', 'really', 'very', 'convenient'] #this is an iterable: something we can implement over, anything we can use as a target of a for loop is iterable 

In [19]:
#iterator is a specific type of thing, its an API that a class can conform to, its a particular type of class 
it = iter(l) # how do we get the iterator of what we want to iterate over, gives an iterator from an argument that is iterable 

In [9]:
type(l), type(it) #type: list_iterator

(list, list_iterator)

In [20]:
# once it is of type iterator, we can call next on it 
next(it) #returns iteration, which is the first string of the list, each time i run will get the next element, and errors out after you finish the list of StopIteration 

#next gives you the next element from the underlying iterable, once you use the iterator up, its up


'iteration'

In [22]:
# putting it all together
# this is essentially what a for loop is (rn an infinite loop) (nvm added a trycatch)
# thus if we can implement this, we can build new things that for can iterate over
it = iter(l)
while True:
    try:
        x = next(it)
        print(x)
    except StopIteration:
        break

iteration
is
really
very
convenient


In [24]:
# iter & next are based on the __iter__ and __next__ special methods
it = l.__iter__()
while True:
    try:
        x = it.__next__()
        print(x)
    except StopIteration:
        break

iteration
is
really
very
convenient


## 3. Implementing iterators with classes

In [25]:
class MyIterator:
    def __init__(self, max):
        #values is zero up to max, not including max
        self.max = max
        self.curr = 0

    # the following methods are required for iterator objects
    def __iter__(self):
        return self 
    
    def __next__(self):
        if self.curr < self.max:
            ret = self.curr
            self.curr +=1
            return ret
        else:
            raise StopIteration()

In [26]:
it = MyIterator(10)

In [38]:
next(it)

StopIteration: 

In [39]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


In [40]:
it = MyIterator(10)
for i in it:
    print(i)

0
1
2
3
4
5
6
7
8
9


An iterator is a *one time use object*! I.e., once we've used it to iterate over elements we cannot typically reset or "rewind" iteration. 

In [41]:
it = MyIterator(10)

In [43]:
# try running this cell multiple times
#after fist time: state shows that it has been used up, wont go again
for i in it:
  print(i)

Iterable objects that can be traversed repeatedly return fresh iterators for each traversal.

In [46]:
# what is happening each time we iterate over `l`, below?
#whats going on behind the scene: creating a new iterator each time in a for loop 
l = list(range(10))
total = 0
for _ in range(10):
    for x in l:
        total += x
total

450

In [48]:
l = list(range(10))
total = 0
for _ in range(10):
    it = iter(l) # we obtain and "use up" a new iterator each loop!
    while True:
        try:
            total += next(it)
        except StopIteration:
            break
total

450

For a container type, we need to implement an `__iter__` method that returns an iterator.

In [55]:
import numpy as np

class ArrayListIterator:
    def __init__(self, array_list):
        self.array_list = array_list # this is the underlying iterable
        self.curr_idx = 0 #tracks our position in it 

    def __iter__(self):
        return self

    def __next__(self):
        if self.curr_idx < self.array_list.size:
                ret = self.array_list.data[self.curr_idx]
                self.curr_idx += 1
                return ret
        else:
            raise StopIteration()
        


class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(2 * len(self.data), dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        #returns an instance of another class which we will define above 
        return ArrayListIterator(self) # create and return an iterator

In [56]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [51]:
it = iter(l)

In [57]:
type(it)

__main__.ArrayListIterator

In [60]:
next(it)

In [ ]:
for x in l:
    print(x)

## 4. Implementing iterators with generators

What's a "generator"?

In [61]:
l = [2**x for x in range(10)] #list comprehension
g = (2**x for x in range(10)) # whats a generator?

In [62]:
type(l), type(g)

(list, generator)

In [65]:
# try running this cell repeatedly
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [69]:
# try running this cell repeatedly
for x in g:
    print(x)

In [70]:
#methods exposed 
# see that __iter__ and __next__ are there 
dir(g)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

Note: Generators implement the iterator API! (`__iter__` and `__next__`)

In [73]:
g = (2**x for x in range(10))

In [74]:
g is iter(g) #same, returns itself, same as an Iterator object

True

In [82]:
next(g)

128

In [83]:
%timeit -n 1000 [2**x for x in range(1000)] #for the list

386 µs ± 3.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [84]:
%timeit -n 1000 (2**x for x in range(1000)) #for the generator, these are way more efficient to create, they dont actually contain all the values like ranges 
#its lazily created

296 ns ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [85]:
g = (2**x for x in range(100))

In [86]:
#cant access elements in an index, it is not subscriptable, only exists as you traverse it 
g[10]

TypeError: 'generator' object is not subscriptable

In [88]:
# try running this cell repeatedly
# first round: gives me the answer
# like an iterator, once i loop through it the information is gone
# cant jump around, cant sort, etc
#   can make a list from the generator if need be 
sum(g)

0

A *generator expression* syntactically resembles a list comprehension, and is similar in that it evaluates to an iterable sequence of values. However, a generator does not represent a fully fleshed out collection of values; instead, values are returned only as they are required through the iteration API (i.e., `next`) --- we refer to this as *lazy evaluation*. 

This makes a generator more efficient than a list (since we don't need to keep all values in the sequence around), but generators can't replace lists in all scenarios (e.g., when we need to jump around in the sequence or revisit values).

### Creating generator functions: `yield`

In [89]:
def foo():
    #execute the yield keyword
    yield

In [90]:
foo()
#evaluates to a generator object

<generator object foo at 0x7eff60632260>

In [91]:
type(foo())

generator

In [100]:
def foo():
    print('hello!')
    yield
    print('goodbye!')
#the presence of yield completely changes the nature of a function call, the function is now known as a generator function
#def foo():
#    print('hello!')
#    if False:
#        yield
#    print('goodbye!')
#doesnt even matter if you can never hit the yield execution 

In [95]:
foo()

<generator object foo at 0x7eff60632880>

In [101]:
g = foo()

In [103]:
next(g)
#run to a yield point, remembers the state,  give control to the caller


goodbye!


StopIteration: 

In [104]:
def foo():
    yield 1
    yield 2
    yield 3

In [105]:
g = foo()
#returns the generator function

In [109]:
next(g)
#runs the part of the fxn until the next yield and remembers its state

StopIteration: 

In [110]:
def countdown(n):
    for x in range(n, 0, -1):
        yield x
    yield 'Blast off!'

In [112]:
for x in countdown(5):
    print(x)

5
4
3
2
1
Blast off!


In [113]:
list(countdown(10))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 'Blast off!']

A *generator function* is a function that contains one or more `yield` statements. When called, a generator function returns a generator object, which effectively allows us to incrementally execute the function using the iteration API. Each call to `next` on the generator will execute the function up to the next `yield` statement; if/when the function completes the generator will raise a `StopIteration` exception (just like an iterator).

### Generators as Data Structure Iterators

In [114]:
class ArrayList:
    def __init__(self):
        self.data = np.empty(1, dtype=object)
        self.size = 0
        
    def append(self, val):
        if self.size == len(self.data):
            ndata = np.empty(len(self.data)*2, dtype=object)
            for i in range(len(self.data)):
                ndata[i] = self.data[i]
            self.data = ndata

        self.data[self.size] = val
        self.size += 1
        
    def __iter__(self):
        #iter now generator fxn
        for i in range(self.size):
            yield self.data[i]

In [115]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [116]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [122]:
# use the now functioning iterator to implement __repr__
class ArrayList(ArrayList):
    def __repr__(self):
        return '['+','.join(repr(x) for x in self) + ']' #generator

In [123]:
l = ArrayList()
for x in range(10):
    l.append(2**x)
l

[1,2,4,8,16,32,64,128,256,512]